Conteo de palabras usando multiprocessing
===

* Última modificación: Mayo 14, 2022

Datos
---

In [36]:
!rm -rf /tmp/input /tmp/output
!mkdir /tmp/input

In [37]:
%%writefile /tmp/input/text0.txt
Analytics is the discovery, interpretation, and communication of meaningful patterns
in data. Especially valuable in areas rich with recorded information, analytics relies
on the simultaneous application of statistics, computer programming and operations research
to quantify performance.

Organizations may apply analytics to business data to describe, predict, and improve business
performance. Specifically, areas within analytics include predictive analytics, prescriptive
analytics, enterprise decision management, descriptive analytics, cognitive analytics, Big
Data Analytics, retail analytics, store assortment and stock-keeping unit optimization,
marketing optimization and marketing mix modeling, web analytics, call analytics, speech
analytics, sales force sizing and optimization, price and promotion modeling, predictive
science, credit risk analysis, and fraud analytics. Since analytics can require extensive
computation (see big data), the algorithms and software used for analytics harness the most
current methods in computer science, statistics, and mathematics.

Writing /tmp/input/text0.txt


In [38]:
%%writefile /tmp/input/text1.txt
The field of data analysis. Analytics often involves studying past historical data to
research potential trends, to analyze the effects of certain decisions or events, or to
evaluate the performance of a given tool or scenario. The goal of analytics is to improve
the business by gaining knowledge which can be used to make improvements or changes.

Writing /tmp/input/text1.txt


In [39]:
%%writefile /tmp/input/text2.txt
Data analytics (DA) is the process of examining data sets in order to draw conclusions
about the information they contain, increasingly with the aid of specialized systems
and software. Data analytics technologies and techniques are widely used in commercial
industries to enable organizations to make more-informed business decisions and by
scientists and researchers to verify or disprove scientific models, theories and
hypotheses.

Writing /tmp/input/text2.txt


In [40]:
import shutil

for src in [
    '/tmp/input/text0.txt',
    '/tmp/input/text1.txt',
    '/tmp/input/text2.txt',
]:
    for i in range(20000):
        shutil.copy(src, src+'.'+str(i)+'.txt')

Lectura de los archivos por línea usando un iterador
---

In [41]:
import fileinput
import glob
import os

def load_files(file_path):
    # -----------------------------------------------------------------------------------
    def make_iterator_from_single_file(file_path):
        with open(file_path, "rt") as file:
            for line in file:
                yield line
    # -----------------------------------------------------------------------------------
    def make_iterator_from_multiple_files(file_path):
        file_path = os.path.join(file_path, "*")
        files = glob.glob(file_path)
        with fileinput.input(files=files) as file:
            for line in file:
                yield line
    # -----------------------------------------------------------------------------------
    if os.path.isfile(file_path):
        return make_iterator_from_single_file(file_path)
    return make_iterator_from_multiple_files(file_path)


Preprocesamiento
---

In [42]:
import string


def tolower(line):
    return line.lower()


def remove_punctuation(line):
    return line.translate(str.maketrans("", "", string.punctuation))


def remove_newline(line):
    return line.replace("\n", "")

Aproximación convencional
---

In [43]:
from collections import defaultdict

In [44]:
%%timeit

counter = defaultdict(int)

for line in load_files('/tmp/input/'):
    
    line = remove_newline(line)
    line = tolower(line)
    line = remove_punctuation(line)
    
    for word in line.split():
        counter[word] += 1
        
  
#
# counter:
# defaultdict(int,
#             {'data': 9,
#              'analytics': 20,
#              'da': 1,
#              'is': 3,
#              'the': 12,
#              'process': 1,
#              'of': 8,
#              'examining': 1,
#              'sets': 1,
#              'in': 5,
#              'order': 1,
#               ...
#

KeyboardInterrupt: 

Aproximación con multiprocessing
---

In [22]:
from toolz.functoolz import compose

def emiter(line):
    return [(word, 1)  for word in line.split()]

def reducer(item):
    return item[0], sum(item[1])

# compose toma las funciones en orden inverso
transformer = compose(
    emiter,
    remove_punctuation,
    tolower,
    remove_newline,
)

In [23]:
from multiprocessing import Queue, Pool

In [33]:
%%timeit

with Pool() as pool:
    
    result = pool.map(transformer, load_files('/tmp/input/'))
    result = [term  for sublist in result for term in sublist]
    
    shuffler = defaultdict(list)
    for key, value in result:
        shuffler[key].append(value)    
        
    result = pool.map(reducer, shuffler.items())

3.82 s ± 372 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
